# # Coronavirus 2020 - effectiveness of measures taken - Spain

In [1]:
import datetime
import os
import time
import joblib
import numpy as np
import pandas as pd

# choose font - can be deactivated
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Inconsolata']
# need many figures for index.ipynb and germany.ipynb
rcParams['figure.max_open_warning'] = 50

import matplotlib.pyplot as plt
plt.style.use('ggplot')

# suppress warning
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

LW = 3   # line width

# set up joblib memory to avoid re-fetching files
joblib_location = "./cachedir"
joblib_memory = joblib.Memory(joblib_location, verbose=0)

%config InlineBackend.figure_formats = ['svg']
%matplotlib inline
# Alternative plotting backend for interative data exploration
# %matplotlib notebook

from coronavirus import overview, fetch_data_germany, germany_get_region

# If you want to edit the source in the notebook, try "%load coronavirus.py" 
# and comment out the import statement above.

In [2]:
def clear_cache():
    """Need to run this before new data for the day is created"""
    joblib_memory.clear()

In [3]:
spanish_regions = ["Andalucía", "Aragón", "Asturias", "Cantabria", "Ceuta", "Castilla y León",
                   "Castilla-La Mancha", "Canarias", "Cataluña", "Extremadura", "Galicia",
                   "Islas Baleares", "Murcia", "Com. Madrid", "Melilla", "Navarra", "País Vasco",
                   "La Rioja", "Com. Valenciana"]

In [4]:
def rename_columns(spanish_data):
    """Rename columns for non-spanish speakers. """
    return spanish_data.rename(columns={'CCAA': 'Admin. region code',
                                        'FECHA': 'Date',
                                        'CASOS': 'Cases',
                                        'Hospitalizados': 'Hospitalized',
                                        'UCI': 'ICU',
                                        'Fallecidos': 'Deceases',
                                        'Recuperados': 'Recovered'}, inplace=True)

In [5]:
#@joblib_memory.cache
def fetch_data_last_execution():
    return datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")

#@joblib_memory.cache
def fetch_data_spain():
    """Data source is https://covid19.isciii.es. The text on the webpage implies that 
    the data comes from the Minitry of Health. """

    datasource = "https://covid19.isciii.es/resources/serie_historica_acumulados.csv"
    t0 = time.time()
    print(f"Please be patient - downloading data from {datasource} ...")
    spain = pd.read_csv(datasource, encoding="ISO-8859-1", engine="python", skipfooter=4)
    rename_columns(spain)
    delta_t = time.time() - t0
    print(f"Completed downloading {len(spain)} rows in {delta_t:.1f} seconds.")

    g2 = spain.set_index(pd.to_datetime(spain['Date']))
    g2.drop(columns=['Date'],inplace=True)
    g2.index.name = 'date'
    last_day = g2.index[-1]
    sel = g2.index == last_day
    cleaned = g2.drop(g2[sel].index, inplace=False)
    fetch_data_last_execution()
    return cleaned

def map_regions(spanish_data):
    acronyms = sorted(set(spanish_data['Admin. region code']))
    regions = dict(zip(acronyms, spanish_regions))
    spanish_data['Region'] = spanish_data['Admin. region code'].map(regions)
    spanish_data.drop(columns=['Admin. region code'], inplace=True)
    return spanish_data

In [6]:
def spain_get_region(region=None):
    spain = fetch_data_spain()
    spain = map_regions(spain)
    """Returns two time series: (cases, deaths)"""
    assert region, "Need to provide a value for the administrative region"
    
    if region:
        assert region in spain['Region'].values, \
            f"{region} not in available Spanish Administrative Regions. These are {sorted(spain['Region'].drop_duplicates())}"

        region = spain[spain['Region'] == region]

        # group over multiple rows for the same region
        cases = region["Cases"]
        cases.country = f'Spain-{region}'
        cases.label = 'cases'

        deaths = region["Deceases"]
        deaths.country = f'Spain-{region}'
        deaths.label = 'deaths'

        return cases, deaths

In [7]:
cases, deaths = spain_get_region(region="Andalucía")

Please be patient - downloading data from https://covid19.isciii.es/resources/serie_historica_acumulados.csv ...
Completed downloading 1104 rows in 0.3 seconds.
